In [2]:
%matplotlib qt

from numpy import *
import scipy.linalg as linalg
from matplotlib.pyplot import *
import pickle
from quadcopter.util import *

rcParams['figure.figsize'] = (10, 6)
rcParams['figure.dpi'] = 160
rcParams['pgf.rcfonts'] = False
rcParams['lines.linewidth'] = 1.2
rcParams['figure.autolayout'] = True

# Replace these paths with the appropriate directories
plot_dir = r'C:\example\plots'
result_dir = r'C:\example\simulation_results'


colors = {
    'reference': 'black',
    'ekf': 'red',
    'mhe1': ['green', 'olive', 'jungle green'],
    'mhe2': ['blue', 'amethyst', 'water blue']
}


def save_plot(plot_name):
    savefig(f"{plot_dir}\\{plot_name}.pgf")


def load_results(file_name):
    complete_path = f"{result_dir}\\{file_name}.pmr"
    with open(complete_path, "rb") as f:
        return pickle.load(f)['results']
    

class ExperimentData:
    def __init__(self, ex_id, raw):
        self.ex_id = ex_id
        self.raw = raw
    
    @property
    def time(self):
        return self.raw['time']
   
    @property 
    def solver(self):
        return self.raw['Solver'].T
    
    @property
    def observer(self):
        return self.raw['Observer'][:, :2].T
    
    @property
    def covariance(self):
        return self.raw['Observer'][:, 2:].T
    
    @property
    def measurement(self):
        return self.raw['Model'].T
    
    
def load_experiment(ex_id, resample_steps=None, start_time=None, end_time=None) -> ExperimentData:
    raw = load_results(ex_id)
    
    start_index = None
    end_index = None
    
    if start_time is not None:
        start_index = time_to_index(start_time, raw['time'])
        
    if end_time is not None:
        end_index = time_to_index(end_time, raw['time'])
    
    raw['time'] = raw['time'][start_index:end_index:resample_steps]
    raw['Solver'] = raw['Solver'][start_index:end_index:resample_steps]
    raw['Observer'] = raw['Observer'][start_index:end_index:resample_steps]
    raw['Model'] = raw['Model'][start_index:end_index:resample_steps]
    return ExperimentData(ex_id, raw)

In [20]:
ekf = load_experiment('ekf')
ekf_clipped = load_experiment('ekf_zero_clipping')
mhe = load_experiment('mhe')

figure(figsize=(6.5, 4))
ref_line, = plot(ekf.time, ekf.solver[0], color=colors['reference'])
plot(ekf.time, ekf.solver[1], color=colors['reference'])

ekf_line, = plot(ekf.time, ekf.observer[0], color=colors['ekf'], linestyle='--')
plot(ekf.time, ekf.observer[1], color=colors['ekf'], linestyle='--')

mhe_line, = plot(mhe.time, mhe.observer[0], color=colors['mhe1'][0], linestyle='--')
plot(mhe.time, mhe.observer[1], color=colors['mhe1'][0], linestyle='--')

xlabel(r"$t$ (s)")
ylabel(r"$p_i$")

annotate('A', xy=(6.5, -0.5))
annotate('B', xy=(6.5, 3))

figlegend((ref_line, ekf_line, mhe_line),
          ('Referenz', 'EKF', 'MHE'))

grid()
save_plot('chemical_partial_pressures')



figure(figsize=(6.5, 4))
ref_line, = plot(ekf_clipped.time, ekf_clipped.solver[0], color=colors['reference'])
plot(ekf_clipped.time, ekf_clipped.solver[1], color=colors['reference'])

ekf_line, = plot(ekf_clipped.time, ekf_clipped.observer[0], color=colors['ekf'], linestyle='--')
plot(ekf_clipped.time, ekf_clipped.observer[1], color=colors['ekf'], linestyle='--')

xlabel(r"$t$ (s)")
ylabel(r"$p_i$")

annotate('A', xy=(45, 0.5))
annotate('B', xy=(45, 3))

figlegend((ref_line, ekf_line),
          ('Referenz', 'EKF mit positiver Begrenzung'))

grid()
save_plot('chemical_ekf_zero_clipping')



figure()
plot(ekf.time, ekf.solver[0] + ekf.solver[1], color=colors['reference'])
plot(ekf.time, ekf.measurement[0], 's')
plot(ekf.time, ekf.observer[0] + ekf.observer[1], color=colors['ekf'])
plot(mhe.time, mhe.observer[0] + mhe.observer[1], color=colors['mhe1'][0])
show()

C:\dev\Python\studienarbeit\lib\site-packages\matplotlib\font_manager.py:1328: UserWarning: findfont: Font family ['lmodern'] not found. Falling back to DejaVu Sans
  (prop.get_family(), self.defaultFamily[fontext]))
